## 1. Traditional RAG
### Retrieveal Augmented Generation

### 1.1 Data Ingection

In [120]:
from langchain_core.documents import Document

In [121]:
doc= Document(
    page_content= "This is the main text content I am using to create RAG",
    metadata= {
        "source":"example.txt",
        "pages":1,
        "auther":"shabnam",
        "date_created": "2025-10-12"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'auther': 'shabnam', 'date_created': '2025-10-12'}, page_content='This is the main text content I am using to create RAG')

In [122]:
## create a smple txt file

import os
os.makedirs("../data/test_files", exist_ok=True)

In [123]:
sample_texts={
    "../data/test_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/test_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}
for filepath, content in sample_texts.items():
    with open (filepath,'w', encoding="utf-8") as f:
        f.write(content)
print ("sample text files created")

sample text files created


### 1.2. Data Parsing (Loader)

In [ ]:
### Text Loader
from langchain_community.document_loaders import TextLoader
loader= TextLoader("../data/test_files/python_intro.txt", encoding="utf-8")
document = loader.load()
print(document)


[Document(metadata={'source': '../data/test_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [125]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader
dir_loader= DirectoryLoader (
    "../data/test_files",
    glob = "**/*.txt",
    loader_cls= TextLoader,
    loader_kwargs={'encoding': 'utf-8'},
    show_progress= False
)
documents= dir_loader.load()
documents

[Document(metadata={'source': '../data/test_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'),
 Document(metadata={'source': '../data/test_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervise

In [144]:
### Directory Loader
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
dir_loader= DirectoryLoader (
    "../data/pdf",
    glob = "**/*.pdf",
    loader_cls= PyMuPDFLoader,
    show_progress= False
)
pdf_documents= dir_loader.load()


### 1.3. Chunking

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

 ### 1.4. Embeding

In [143]:
### embeding and vector
import numpy as np
from sentence_transformers import SentenceTransformer
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity
import chromadb


In [128]:
class EmbeddingManager:
    def __init__(self, model_name :str = "all-MiniLM-L6-v2"):
        self.model_name= model_name
        self.model = None
        self._load_model()
    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model= SentenceTransformer(self.model_name)
            print(f"Model loaded successfully.Embedding dimenssion : {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model: {self.model_name}:{e}")
            raise
    def generate_embedding(self, texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model not loaded")
        print (f"Generating embedding for {len(texts)} texts...")
        embedding = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embedding with shape {embedding.shape}")
        return embedding
    def get_embedding_dimenssion(self) ->int:
        if not self.model:
            raise ValueError("Model not Loaded")
        return self.model.get_sentence_embedding_dimension()

##iinitiallize embedding mamanger
embedding_manager = EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully.Embedding dimenssion : 384


### 1.5. VectorStore

In [129]:
### VectorStore
import chromadb
class VectoreSore:
    def __init__(self, collection_name:str ="pdf_docyments",persist_dictionary: str = "../data/vector_store"):
        self.collection_name= collection_name
        self.persist_dictionary = persist_dictionary
        self.client = None
        self.collection= None
        self._initialise_store()
    
    def _initialise_store(self):
        try:
            #create persist ChromaDB client
            os.makedirs(self.persist_dictionary, exist_ok=True)
            self.client = chromadb.PersistentClient (path= self.persist_dictionary)

            #Get or Create Collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description": "PDF document embedding for RAG"}
            )
            print(f"Vectore store initialized.Collection :{self.collection_name}")
            print(f"Existing documents in Collection :{self.collection.count()}")
        
        except Exception as e:
            print(f"Error Initializing vectore store : {e}")
            raise
    
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError ("Number of documents must match the number of embeddings") 
        print(f"Adding {len(documents)} documents to vectore store")

        #prepare data for Chromadb
        ids = []
        metadatas = []
        documents_text = []
        embedding_list= []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            #Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #Prepare meradata
            metadata = dict(doc.metadata)
            metadata['doc_index']= i
            metadata['content_length']= len(doc.page_content)
            metadatas.append(metadata)

            #Document Content
            documents_text.append(doc.page_content)

            #Embedding
            embedding_list.append(embedding.tolist())
        
        #Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings= embedding_list,
                metadatas= metadatas,
                documents= documents_text
            )
            print (f"Successfully added {len(documents)} documents to vectore store")
            print (f" Total documents in collection {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vectore store : {e}")
            raise

vectorestore= VectoreSore()


Vectore store initialized.Collection :pdf_docyments
Existing documents in Collection :84


In [ ]:
### Read all the pdf's inside the directory
from pathlib import Path

def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

In [ ]:
### Text splitting get into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [ ]:
chunks=split_documents(all_pdf_documents)

In [133]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

## Generate THE Embedding
embedding= embedding_manager.generate_embedding(texts)

### Store in the Vectore database
vectorestore.add_documents(chunks, embedding)


Generating embedding for 42 texts...


Batches: 100%|██████████| 2/2 [00:00<00:00,  6.94it/s]

Generated embedding with shape (42, 384)
Adding 42 documents to vectore store
Successfully added 42 documents to vectore store
 Total documents in collection 126


### 1.6. Retriver Pipeline VectoreStore

In [134]:
class RAGRetriver :
    """Handle query-based retrival from the vectore store"""

    def __init__(self, vector_estore:VectoreSore, embedding_manager:EmbeddingManager):
        """initialize the retriver
        
        Args: 
        vectore_store: Vectore store containing document embeddings
        embedding_manager: Manager for generating query embedings
        """
        
        self.vector_estore = vector_estore
        self.embedding_manager= embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold:float = 0.0) -> List[Dict[str,Any]]:
        """
        Retrive relevant documents for a query

        Args: 
            query; The search query
            top_k: Number of top reslts to return
            score threshold : Minimum similarity scire threshold

        Returns:
            List of dictionaries containing retrived documents and metadatas
        """

        print(f"Retriving documents for query : '{query}'")
        print(f"Top_k : {top_k}, Score threshold {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embedding([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_estore.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []
rag_retriver= RAGRetriver(vectorestore,embedding_manager)

In [135]:
rag_retriver

In [ ]:
rag_retriver.retrieve("Ingénieur QoS FixeBOUYGUES TELECOM - Depuis 2019?") 

### 1.7. Generation
### Integration Vectoredb Context With LLM Output

In [137]:
## Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key= os.getenv("GROQ_API_KEY")

## Initialize the Groq LLM
llm= ChatGroq(groq_api_key= groq_api_key, model_name= 'groq/compound-mini', temperature= 0.1, max_tokens=1024)

#Simple RAG Function
def rag_simple(query, retriever, llm, top_k=3):
    results= retriever.retrieve(query, top_k= top_k)
    context = "".join([doc['content']for doc in results]) if results else ""
    if not context: 
        return "No relevant context found to answer the questions"
    ## generate the answer using groq llm
    prompt=f"""You are a helpful HR AI assistant. Use the following context to answer the question accurately and concisely.

        Context:
        {context}

        Question: {query}

        Answer: """

    response = llm.invoke([prompt.format(context= context, query= query)])
    return response.content



In [138]:
answer = rag_simple("BOUYGUES TELECOM - Depuis 2019", rag_retriver,llm)
print(answer)

Retriving documents for query : 'BOUYGUES TELECOM - Depuis 2019'
Top_k : 3, Score threshold 0.0
Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 26.59it/s]

Generated embedding with shape (1, 384)
Retrieved 0 documents (after filtering)
No relevant context found to answer the questions


## 2. Advanced RAG

In [ ]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False, stream =True, summarize= True):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output


# Example usage:
result = rag_advanced("Ingénieur QoS FixeBOUYGUES TELECOM - Depuis 2019", rag_retriver, llm, top_k=10, min_score=0.1, return_context=True, stream= True, summarize= True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

In [140]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriver, llm)
result = adv_rag.query("Centres d'intérê", top_k=9, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retriving documents for query : 'Centres d'intérê'
Top_k : 9, Score threshold 0.1
Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 96.67it/s]

Generated embedding with shape (1, 384)
Retrieved 0 documents (after filtering)



Final Answer: No relevant context found.
Summary: The response indicates that there is no pertinent information available. Consequently, no context can be provided.
History: {'question': "Centres d'intérê", 'answer': 'No relevant context found.', 'sources': [], 'summary': 'The response indicates that there is no pertinent information available. Consequently, no context can be provided.'}
